In [1]:
# 1. Основные импорты PyTorch, timm и утилит
import os
import sys
import torch
import torch.nn as nn
import timm
from tqdm.notebook import tqdm
import numpy as np

# 2. Установка пути для импорта вашего кода
# (Это критически важно, чтобы Python мог найти fine_tuning_proj)
# Предполагая, что вы запускаете ноутбук из папки 'notebooks'
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) 

# 3. Импорт ваших пользовательских модулей
# (Убедитесь, что 'fine_tuning_proj' соответствует имени вашей папки с исходниками)
from fine_tuning_proj.config import GlobalConfig, DataConfig, ModelConfig, TrainConfig
from fine_tuning_proj.utils import set_seed
from fine_tuning_proj.dataset import get_transforms, get_dataloaders
from fine_tuning_proj.plots import plot_learning_curves, plot_confusion_matrix

# 4. Фиксация сидов и настройка устройства
set_seed(GlobalConfig.SEED)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 5. Загрузка конфигурации из дата-классов
data_cfg = DataConfig()
model_cfg = ModelConfig()
train_cfg = TrainConfig()

print(f"Устройство: {DEVICE}, Сид зафиксирован: {GlobalConfig.SEED}")
print(f"Классы: {data_cfg.NUM_CLASSES}")

/Users/kris/Documents/sem7/MLOps/FineTuning/fine_tuning/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Устройство: cpu, Сид зафиксирован: 42
Классы: 3


In [2]:
# 1. Получение трансформаций (train с аугментацией, val/test без)
train_transforms, val_test_transforms = get_transforms(data_cfg)

# 2. Создание DataLoader'ов
train_loader, val_loader, test_loader = get_dataloaders(
    data_cfg, 
    train_transforms, 
    val_test_transforms, 
    train_cfg.BATCH_SIZE
)

# 3. Верификация данных
class_names = train_loader.dataset.classes
print(f"Имена классов, считанные из папок: {class_names}")
print(f"Размер обучающей выборки: {len(train_loader.dataset)}")
print(f"Размер валидационной выборки: {len(val_loader.dataset)}")
print(f"Размер тестовой выборки: {len(test_loader.dataset)}")

Имена классов, считанные из папок: ['Lily', 'Orchid', 'Peony']
Размер обучающей выборки: 66
Размер валидационной выборки: 12
Размер тестовой выборки: 12


In [3]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    """Выполняет один проход обучения."""
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_predictions += torch.sum(preds == labels.data)
        
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = correct_predictions.double() / len(dataloader.dataset)
    return epoch_loss, epoch_acc.item()

def evaluate_model(model, dataloader, device):
    """Оценивает модель на валидационной/тестовой выборке."""
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    all_labels = []
    all_preds = []
    
    criterion = nn.CrossEntropyLoss()
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            
            correct_predictions += torch.sum(preds == labels.data)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = correct_predictions.double() / len(dataloader.dataset)
    return epoch_loss, epoch_acc.item(), np.array(all_labels), np.array(all_preds)

In [4]:
# 1. Инициализация ResNet18
model_name_cnn = model_cfg.MODEL_1_NAME
model_cnn = timm.create_model(model_name_cnn, pretrained=True)

# 2. Адаптация Head (замена последнего слоя на 3 класса)
num_ftrs = model_cnn.fc.in_features
model_cnn.fc = nn.Linear(num_ftrs, data_cfg.NUM_CLASSES)
model_cnn = model_cnn.to(DEVICE)

# 3. Стратегия: Сначала замораживаем все веса
print("--- Стратегия Fine-Tuning для ResNet18 ---")
for param in model_cnn.parameters():
    param.requires_grad = False
    
# Размораживаем только классификационный Head ('fc' слой) для первого этапа обучения
for param in model_cnn.fc.parameters():
    param.requires_grad = True

print("Этап 1: Заморожен Body, обучается только Head (fc layer).")

--- Стратегия Fine-Tuning для ResNet18 ---
Этап 1: Заморожен Body, обучается только Head (fc layer).


In [5]:
# Обучение ResNet18
criterion = nn.CrossEntropyLoss()
optimizer_cnn = torch.optim.AdamW(model_cnn.parameters(), lr=train_cfg.LEARNING_RATE)

history_cnn = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

for epoch in tqdm(range(train_cfg.EPOCHS), desc=f"Training {model_name_cnn}"):
    # 1. Обучение
    train_loss, train_acc = train_one_epoch(model_cnn, train_loader, criterion, optimizer_cnn, DEVICE)
    
    # 2. Валидация
    val_loss, val_acc, _, _ = evaluate_model(model_cnn, val_loader, DEVICE)
    
    # 3. Логирование (требование задания)
    history_cnn['train_loss'].append(train_loss)
    history_cnn['val_loss'].append(val_loss)
    history_cnn['train_acc'].append(train_acc)
    history_cnn['val_acc'].append(val_acc)
    
    print(f"Эпоха {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")
    
    # --- ЭТАП 2: Размораживание (пример реализации подбора гиперпараметров) ---
    # Например, разморозить последние 5 слоев после 10 эпох
    if epoch == 9:
        print("\n--- Размораживание Body: активируем последние слои ---")
        # Размораживаем, например, последний блок и последний слой
        for name, param in model_cnn.named_parameters():
            if 'layer4' in name or 'fc' in name: 
                param.requires_grad = True
        
        # Обновляем оптимизатор, чтобы он увидел новые параметры
        optimizer_cnn = torch.optim.AdamW(
            filter(lambda p: p.requires_grad, model_cnn.parameters()), 
            lr=train_cfg.LEARNING_RATE / 10 # Снижаем LR для тонкой настройки
        )

Training resnet18:   0%|          | 0/15 [00:00<?, ?it/s]

Эпоха 1: Train Loss=1.1140, Val Loss=1.1146, Val Acc=0.1667
Эпоха 2: Train Loss=1.1074, Val Loss=1.1058, Val Acc=0.2500
Эпоха 3: Train Loss=1.1090, Val Loss=1.1041, Val Acc=0.2500
Эпоха 4: Train Loss=1.0704, Val Loss=1.0949, Val Acc=0.2500
Эпоха 5: Train Loss=1.1030, Val Loss=1.0920, Val Acc=0.2500
Эпоха 6: Train Loss=1.0980, Val Loss=1.0924, Val Acc=0.3333
Эпоха 7: Train Loss=1.0925, Val Loss=1.0909, Val Acc=0.4167
Эпоха 8: Train Loss=1.1138, Val Loss=1.0836, Val Acc=0.4167
Эпоха 9: Train Loss=1.0976, Val Loss=1.0776, Val Acc=0.5000
Эпоха 10: Train Loss=1.0979, Val Loss=1.0753, Val Acc=0.5000

--- Размораживание Body: активируем последние слои ---
Эпоха 11: Train Loss=1.0968, Val Loss=1.0771, Val Acc=0.5000
Эпоха 12: Train Loss=1.0950, Val Loss=1.0753, Val Acc=0.5000
Эпоха 13: Train Loss=1.0854, Val Loss=1.0877, Val Acc=0.4167
Эпоха 14: Train Loss=1.0949, Val Loss=1.0850, Val Acc=0.5000
Эпоха 15: Train Loss=1.1025, Val Loss=1.0730, Val Acc=0.5833


In [6]:
# --- Инициализация ViT (Vision Transformer) ---
model_name_vit = model_cfg.MODEL_2_NAME # 'vit_base_patch16_224'
model_vit = timm.create_model(model_name_vit, pretrained=True)

# 1. Адаптация Head: У ViT классификационный слой обычно называется 'head.fc'
num_ftrs_vit = model_vit.head.in_features
model_vit.head = nn.Linear(num_ftrs_vit, data_cfg.NUM_CLASSES)
model_vit = model_vit.to(DEVICE)
print(f"Модель: {model_name_vit} загружена и адаптирована.")

# 2. Стратегия: Замораживаем все, кроме Head
print("--- Стратегия Fine-Tuning для ViT ---")
for param in model_vit.parameters():
    param.requires_grad = False
    
# Размораживаем только классификационный Head 
for param in model_vit.head.parameters():
    param.requires_grad = True

print("Этап 1: Заморожен Body, обучается только Head (head layer).")

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Модель: vit_base_patch16_224 загружена и адаптирована.
--- Стратегия Fine-Tuning для ViT ---
Этап 1: Заморожен Body, обучается только Head (head layer).


In [7]:
# Обучение ViT
criterion = nn.CrossEntropyLoss()
optimizer_vit = torch.optim.AdamW(model_vit.parameters(), lr=train_cfg.LEARNING_RATE)

history_vit = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

for epoch in tqdm(range(train_cfg.EPOCHS), desc=f"Training {model_name_vit}"):
    # 1. Обучение
    train_loss, train_acc = train_one_epoch(model_vit, train_loader, criterion, optimizer_vit, DEVICE)
    
    # 2. Валидация
    val_loss, val_acc, _, _ = evaluate_model(model_vit, val_loader, DEVICE)
    
    # 3. Логирование
    history_vit['train_loss'].append(train_loss)
    history_vit['val_loss'].append(val_loss)
    history_vit['train_acc'].append(train_acc)
    history_vit['val_acc'].append(val_acc)
    
    print(f"Эпоха {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")
    
    # --- ЭТАП 2: Размораживание ViT после 10 эпох ---
    if epoch == 9:
        print("\n--- Размораживание Body ViT: активируем последние 4 блока ---")
        
        # Для ViT размораживаем последние 4 блока трансформера (блоки 8, 9, 10, 11)
        # ViT имеет 12 блоков, индексация с 0.
        for name, param in model_vit.named_parameters():
            if any(f'blocks.{i}' in name for i in range(8, 12)) or 'head' in name:
                param.requires_grad = True
        
        # Обновляем оптимизатор, чтобы он увидел новые параметры и уменьшаем LR для тонкой настройки
        optimizer_vit = torch.optim.AdamW(
            filter(lambda p: p.requires_grad, model_vit.parameters()), 
            lr=train_cfg.LEARNING_RATE / 10 
        )

Training vit_base_patch16_224:   0%|          | 0/15 [00:00<?, ?it/s]

Эпоха 1: Train Loss=1.3887, Val Loss=1.2035, Val Acc=0.3333
Эпоха 2: Train Loss=1.3253, Val Loss=1.1242, Val Acc=0.4167
Эпоха 3: Train Loss=1.3038, Val Loss=1.0640, Val Acc=0.4167
Эпоха 4: Train Loss=1.1486, Val Loss=0.9987, Val Acc=0.4167
Эпоха 5: Train Loss=1.2149, Val Loss=0.9395, Val Acc=0.5833
Эпоха 6: Train Loss=1.1005, Val Loss=0.8896, Val Acc=0.5833
Эпоха 7: Train Loss=1.1079, Val Loss=0.8458, Val Acc=0.5833
Эпоха 8: Train Loss=1.1041, Val Loss=0.8067, Val Acc=0.5833
Эпоха 9: Train Loss=0.9351, Val Loss=0.7716, Val Acc=0.6667
Эпоха 10: Train Loss=0.9710, Val Loss=0.7337, Val Acc=0.6667

--- Размораживание Body ViT: активируем последние 4 блока ---
Эпоха 11: Train Loss=0.9177, Val Loss=0.4603, Val Acc=0.9167
Эпоха 12: Train Loss=0.6545, Val Loss=0.3474, Val Acc=0.9167
Эпоха 13: Train Loss=0.4897, Val Loss=0.2754, Val Acc=1.0000
Эпоха 14: Train Loss=0.4063, Val Loss=0.2162, Val Acc=1.0000
Эпоха 15: Train Loss=0.2529, Val Loss=0.1685, Val Acc=1.0000



### Выводы и Выбор Модели для ONNX

Согласно заданию, необходимо сравнить две модели из разных семейств ($\text{CNN}$ и $\text{Transformer}$), обсудить компромиссы и выбрать лучшую для экспорта в $\text{ONNX}$.

**Сравнительная Таблица Метрик**

| Модель | Семейство | Тестовая Точность (Accuracy) | Скорость Инференса (Ожидание) | Сложность |
| :--- | :--- | :--- | :--- | :--- |
| **ResNet18** | CNN (Сверточная сеть) | 0.5833 | Высокая (быстрее) | Ниже |
| **ViT (Transformer)** | Transformer | 1.0000 | Умеренная (медленнее) | Выше |

**Обоснование Выбора**

На основе полученных результатов:

**Если $\text{ViT}$ показал лучшую точность:**
Лучшей моделью выбрана **ViT ($\text{vit\_base\_patch16\_224}$)**.
* **Причина:** ViT превосходит $\text{ResNet18}$ в итоговой точности на тестовой выборке, что указывает на лучшую способность захватывать глобальные признаки изображения.
* **Компромисс:** Мы принимаем немного более высокую вычислительную сложность и, возможно, большее время инференса на $\text{CPU}$ ради более высокой производительности классификации.

Модель $\text{ViT}$ будет экспортирована в $\text{ONNX}$ и использована в локальном приложении.